# Data Visualization Final Project Part 2: The World Happiness Report Dashboard

**Group BQ-Alots: Ankita Pant, Archana Kulkarni, Sam Walkow, and Zoë Kaler**

**About Our Data**
- Welcome to World Happiness Report Dashboard! We have data from 2015, 2016, and 2017, each with around 155 rows, each row representing a country. The columns have factors such as happiness rank, happiness score, GDP per capita, family, life expectancy, government corruption. These seven are consistent through each data set and they total the country’s happiness score: GDP per capita, family, life expectancy, generosity, government corruption and dystopia residual. The happiness rank corresponds to the score. Data source: https://www.kaggle.com/unsdsn/world-happiness
            

In [1]:
%matplotlib inline
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
import bqplot
from bqplot.market_map import MarketMap

In [2]:
df_2015 = pd.read_csv('2015.csv')
df_2016 = pd.read_csv('2016.csv')
df_2017 = pd.read_csv('2017.csv')

In [3]:
country_names = df_2017['Country']
country_rank = df_2017['Happiness.Rank']
economy = df_2017["Economy..GDP.per.Capita."]
family = df_2017['Family']
health = df_2017["Health..Life.Expectancy."]
freedom = df_2017["Freedom"]
generosity = df_2017["Generosity"]
trust = df_2017["Trust..Government.Corruption."]
dystopia = df_2017['Dystopia.Residual']

# Part 1: 2015, 2016, 2017 Happiness Score Counts 
**This visualization gives an overview of the distribution of the happiness score overtime. This is the only visualization that incorporates all three years of data; 2017 will be used for the rest of the visualizations. We choose 2017 because variables were not consistent throughout all three years and 2017 tells the most recent story.**

In [4]:
# Scales and Axis

x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()
col = bqplot.ColorScale(scheme='plasma')
ax_c = bqplot.ColorAxis(scale=col)
ax_x = bqplot.Axis(scale=x_sc, tick_format='0.1f', label = 'Happiness Score')
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical', label = 'Count')

# 2015 Hist

z_data = df_2015["Happiness Score"]
zhist = bqplot.Hist(sample=z_data, scales={'sample': x_sc, 'count': y_sc}, colors = ['#e8705b'])
zfig = bqplot.Figure(marks=[zhist], axes=[ax_x, ax_y], padding_y=0, 
                    title = "2015 Happiness Score Count")

# 2016 Hist

y_data = df_2016["Happiness Score"]
yhist = bqplot.Hist(sample=y_data, scales={'sample': x_sc, 'count': y_sc}, colors = ['#e8705b'])
yfig = bqplot.Figure(marks=[yhist], axes=[ax_x, ax_y], padding_y=0, 
                    title = "2016 Happiness Score Count", color = col)


# 2017 Hist
x_data = df_2017["Happiness.Score"]
hist = bqplot.Hist(sample=x_data, scales={'sample': x_sc, 'count': y_sc}, colors = ['#e8705b'])
fig = bqplot.Figure(marks=[hist], axes=[ax_x, ax_y], padding_y=0, 
                    title = "2017 Happiness Score Count")

# set up slider

islider = ipywidgets.IntSlider(min= 8, max=50, step = 1, title = 'Bin Number')
ipywidgets.link((islider, "value"),(hist, "bins"))
ipywidgets.link((islider, "value"),(yhist, "bins"))
ipywidgets.link((islider, "value"),(zhist, "bins"))

display(ipywidgets.VBox([ipywidgets.Label(value="Number of Bins"),islider]))

display(ipywidgets.HBox([zfig, yfig, fig]))

# Part 2: Countries According to 2017 Happiness Score
**This visualization shows at a glance where countries stand in happiness rank. You can also see each country’s happiness score breakdown. We choose to put this information in the tooltip so you can dig a little deeper into each country and begin to understand how their score is made up and what that might mean for the lives in that country. We choose to start with the happiest because that places countries in consecutive order by score.** 

In [5]:
sc_x = bqplot.LinearScale()
sc_y = bqplot.LinearScale()

ax_x = bqplot.Axis(scale=sc_x, grid_lines='dashed')
ax_y = bqplot.Axis(scale=sc_y, orientation='vertical', grid_lines='dashed')


col = bqplot.ColorScale(scheme='plasma', reverse = True)
ax_c = bqplot.ColorAxis(scale=col, label='Unhappy to Happy', visible=True, reverse = True)
market_map = MarketMap(names= country_names,
                       ref_data= df_2017, 
                       tooltip_fields= (['Country', 'Economy..GDP.per.Capita.', 'Family',
                                       'Health..Life.Expectancy.', 'Freedom', 'Generosity', 
                                       'Trust..Government.Corruption.','Happiness.Score', 'Happiness.Rank']),
                       color= country_rank,
                       layout={'min_width': '1000px', 'min_height': '800px'}, scales={'color': col},
                       axes = [ax_c],title = 'Countries According to 2017 Happiness Score')
market_map

MarketMap(axes=[ColorAxis(label='Unhappy to Happy', scale=ColorScale(reverse=True, scheme='plasma'))], color=a…

# Part 3: Happiness Score as a Function of Happiness Factors
**This visualization show the contribution of each factor to happiness score. It is essentially the tooltips from the marketmap visualized by factor. We chose the dropdown because wanted our audience to be able to dynamically experience each correlation of factor and happiness score separately.**

In [6]:
happy_score = df_2017['Happiness.Score']

sc_x = bqplot.LinearScale()
sc_y = bqplot.LinearScale()

line = bqplot.Lines(x = happy_score, y = trust,
            scales = {'x': sc_x, 'y': sc_y}, colors = ["#f9a346"])

ax_x = bqplot.Axis(scale = sc_x, label = 'Happiness Score')
ax_y = bqplot.Axis(scale=sc_y, orientation='vertical', label='Factor Contribution')    

select_mul = ipywidgets.Dropdown(
   options = ['Dystopia', 'Economy', 'Family', 'Freedom', 'Generosity', 'Health', 'Trust'],
    value = 'Dystopia',
   description='Factors',
   disabled=False)
        

def factor_select(change):
    if change['new'] == 'Dystopia':
        line.y = df_2017["Dystopia.Residual"]
    if change['new'] == 'Economy':
        line.y = df_2017["Economy..GDP.per.Capita."]
    if change['new'] == 'Family':
        line.y = df_2017["Family"]
    if change['new'] == 'Freedom':
        line.y = df_2017["Freedom"]
    if change['new'] == 'Generosity':
        line.y = df_2017["Generosity"]
    if change['new'] == 'Health':
        line.y = df_2017["Health..Life.Expectancy."]
    if change['new'] == 'Trust':
        line.y = df_2017["Trust..Government.Corruption."]  


fig_trust = bqplot.Figure(marks=[line], axes=[ax_x, ax_y],
                          title='Happiness Score as a Function of Happiness Factors')

select_mul.observe(factor_select)

display(ipywidgets.VBox([select_mul, fig_trust]))

# Part 4: Factor Breakdown in the Five Most and Least Happy Countries
**This visualization zooms in on the line plot’s story and shows the breakdown for the five top and five bottom countries by rank. We chose the top and bottom five and not all the countries to show the juxtaposition of what factors make up score for counties with opposite rank.**

In [7]:
most_common_rank = df_2017["Happiness.Rank"].value_counts()[:5].index.values

x_ord = bqplot.OrdinalScale()
x_ord1 = bqplot.OrdinalScale()
y_sc = bqplot.LinearScale()

ax_x = bqplot.Axis(scale=x_ord1, label = "Countries", xlim = 5.0)
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f', 
                   label = "Amount Factor Contributed to Happiness Score", ylim = 1.5)

ax_x1 = bqplot.Axis(scale=x_ord, label = "Countries")
ax_y1 = bqplot.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f', 
                   label = "Factor Contribution")

margin = dict(top=80, bottom=50, left=50, right=50)
margin1 = dict(top=50, bottom=50, left=50, right=50)

def_tt = bqplot.Tooltip(fields=['x', 'y'], formats=['', '.2f'])

bar = bqplot.Bars(x= df_2017["Country"][:5], y=[economy, family, health, freedom, generosity, trust], 
           scales={'x': x_ord1, 'y': y_sc}, padding=0.2, type='grouped',
           colors=(["purple", "blue", "maroon", "red", "orange", "yellow"]), 
                  labels=['Economy', 'Family', 'Health', 'Freedom', 'Generosity', 'Trust'],
            stroke = 'white', display_legend=True,
                interactions={'hover': 'tooltip'})

bar.tooltip=def_tt

fig1 = bqplot.Figure(marks=[bar], axes=[ax_x, ax_y], fig_margin = margin, 
                     title = 'Factor Breakdown for Happiness Score in Five Most Happy Countries')

bar1 = bqplot.Bars(x= df_2017["Country"][:149:-1], 
                  y=[economy[::-1], family[::-1], health[::-1], freedom[::-1], generosity[::-1], trust[::-1]], 
           scales={'x': x_ord, 'y': y_sc}, padding=0.2,type='grouped',
           colors=(["purple", "blue", "maroon", "red", "orange", "yellow"]), 
                   labels=['Economy', 'Family', 'Health', 'Freedom', 'Generosity', 'Trust'],
                   display_legend=True, stroke = 'white', interactions={'hover': 'tooltip'})

bar1.tooltip=def_tt

fig2 = bqplot.Figure(marks=[bar1], axes=[ax_x1, ax_y1], fig_margin = margin1, 
                     title = 'Factor Breakdown for Happiness Score in Five Least Happy Countries')

display(ipywidgets.VBox([fig1, fig2]))

## Challenges
- We had three main challenges while creating this visualization. The first was figuring out how to visualize the breakdown of the happiness score. We were fascinated by the seven factors that make up the score and we wanted to dig into that. This is why our dashboard slowly narrows in on this aspect of the data.

- The second main challenge was small details in bqplot. We had trouble moving and editing the legends, editing text, etc. These small details are what we wanted to make this dashboard look perfect, but we were unable to find ways to implement these finishing touches. 

- The last, and most important, challenge was maintaining a sense of empathy for the people in the countries, specifically the non-happy countries. We had to be careful not to reduce someone’s pain into a rank or score. We believe that is dashboard can be a starting place for figuring out why these people may not be as happy as possible and how to make them happier. 
